In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer
from numpy import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<ipython-input-1-e91b8758e284>:11: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  from numpy import math


In [3]:
data = pd.read_csv('../data/online_retail_II.csv')
print(" Data imported successfully!!")
data.head(3).style.set_properties(**{"background-color": "#cd5c5c","color": "black", "border-color": "black"})

 Data imported successfully!!


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950000,13085.000000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750000,13085.000000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750000,13085.000000,United Kingdom


In [6]:
pd.set_option('display.max_columns', None)
def data_overview(df, head = 5):
    print("SHAPE OF DATASET".center(125, '-'))
    print("Row: ".format(df.shape[0]))
    print("Column: ".format(df.shape[1]))
    print("HEAD".center(125,'-'))
    print(df.head(head))
    print("DATA TYPES".center(125,'-'))
    print(df.dtypes.value_counts())
    print("MISSING VALUES".center(125,'-'))
    print(df.isnull().sum()[df.isnull().sum()> 0].sort_values(ascending = False))
    print("DUPLICATE VALUES".center(125,'-')) 
    print(df.duplicated().sum())
    print("STATISTICS OF DATA".center(125,'-'))
    print(df.describe(include="all"))
    print("DATA INFO".center(125,'-'))
    print(df.info())

data_overview(data)

-------------------------------------------------------SHAPE OF DATASET------------------------------------------------------
Row: 
Column: 
-------------------------------------------------------------HEAD------------------------------------------------------------
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3  2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4  2009

#### Point to be noted
* The Dataset has Rows: 1067371 and Columns:8
* The Dataset has 3 types of columns: strings(5), integer(1), float(2)
* The Dataset has Missing values in Customer ID (243007) and Description (4382)
* Invoice starts with the 'c' needs to be cleaned as it is cancelled transaction
* The Dataset has duplicates
* Aslo check for negative value and outliers in Quantity and Price